# HPC as solutions for AI: TensorFlow

<p style='text-align: justify;'>
In this section, it will be shown how to optimize TensorFlow models, accelerating training and execution using GPUs.
</p>    

The principal goals are:
* **Understand** what is TensorFlow,
* **Learn** the basic concepts of TensorFlow for GPUs,
* **Familiarize** yourself with the CIFAR-10 and CIFAR-100 datasets by classifying their various classes,
* **Create** a model using TensorFlow.

## What applications uses TensorFlow in AI?

<p style='text-align: justify;'>
TensorFlow is an open-source machine learning framework developed by Google that is widely used in artificial intelligence (AI) applications. It provides a comprehensive set of tools, libraries, and community support for building and deploying various machine learning models, such as deep learning, computer vision, and neural networks. Overall, TensorFlow is a versatile framework that covers a wide range of machine learning and AI applications. However, when creating a model, the training process becomes a bottleneck as it takes a lot of time, but as we will see throughout the module, TensorFlow allows us to speed up this processing.
</p>    

## The solution: GPUs and TensorFlow

<p style='text-align: justify;'> 
In addition to being a powerful library for machine learning, TensorFlow allows you to train the created model using GPUs, enhancing and accelerating the training process. As we will see later, the performance gain when training a TensorFlow model on a GPU is enormous because GPUs are designed with thousands of processing cores, which allow the execution of many simultaneous operations. This is especially beneficial for matrix calculations, which are fundamental in machine learning algorithms such as neural networks.
</p>

##  ☆ Challenge: Zoo breakout!☆ 

<p style='text-align: justify;'> 
    Recently, an unexpected incident occurred at the local zoo, <b>Orange Grove Zoo</b>: all the animals escaped from their enclosures and are now roaming freely. To deal with this situation, we need your help locating and classifying the escaped animals, distinguishing each animal class, and identifying possible vehicles in the same environment.
</p>
<p style='text-align: justify;'> 
You have been assigned as the person responsible for developing a computer vision system capable of identifying and classifying the escaped animals and identifying the presence of vehicles in the images. We will use the CIFAR-10 dataset and the TensorFlow library to train a deep-learning model for this challenge.
</p>
CIFAR-10 and CIFAR-100 datasets comprehensively collect $32$x$32$ pixel images grouped into $10$ distinct classes.

- [CIFAR-10 Dataset](https://www.cs.toronto.edu/~kriz/cifar.html): CIFAR-10 consists of $60,000$ images, each belonging to one of the ten classes: airplanes, cars, birds, cats, deer, dogs, frogs, horses, ships, and trucks. This dataset offers a diverse set of images representing everyday objects.

- [CIFAR-100 Dataset](https://www.cs.toronto.edu/~kriz/cifar.html): CIFAR-100 expands upon the CIFAR-10 concept, containing 60,000 images as well. However, it introduces a more challenging task by categorizing images into 100 classes. These classes include various subcategories such as fruits, animals, vehicles, and more.

a) **Create** deep neural network model utilizing the TensorFlow library for the classification of animals and vehicles on a GPU environment using the CIFAR-10 dataset.

b) **Conduct** a comparative analysis between models trained on a CPU and GPU to highlight disparities in results.

c) Now, use the CIFAR-100 dataset for the classification of animals and vehicles on a GPU. Would it be a good decision to use a GPU or CPU environment for the training process?

### ☆ Solution for `CIFAR-10` using TensorFlow on GPU ☆

#### ⊗ Importing packages

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import time

#### ⊗ Verify the devices

It is very important, before trying to execute anything on any device, to verify if it is available and if TensorFlow can use it.

#####  Checking the environmental availability

In [ ]:
# Checking if GPU is available
print(tf.config.list_physical_devices('GPU'))

#### ⊗ Downloading the dataset

Now we need to download the CIFAR-10 dataset to be able to make predictions. This dataset is a set of labeled images, meaning that each image already has a known label.

In [ ]:
# Loading the CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

#### ⊗ Normalizing the dataset

After downloading the entire set of images, we need to normalize them so that we can use them in our example.

In [ ]:
# Normalizing pixel values to the [0, 1] range
train_images, test_images = train_images / 255.0, test_images / 255.0

#### ⊗ Training the model

<p style='text-align: justify;'>
 Below we have the training function with model creation and model compilation. Notice that we need to do the creation and compilation together with the training because we need to set in which device everything will be done. We need to do this because if we don't set the device to create and compile the model, the TensorFlow will choose the faster device, in this case, the GPU, so if we try to use the CPU to train the model, it will fail because the model will be created on the GPU. 
</p>

In [ ]:
def train_model(device, train_images, train_labels):
    with tf.device(device):
        
        # Creating the CNN model
        model = models.Sequential([
            layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.Flatten(),
            layers.Dense(64, activation='relu'),
            layers.Dense(10)
        ])

        # Compiling the model
        model.compile(optimizer='adam',
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])

        start_time = time.time()
        history = model.fit(train_images, train_labels, epochs=10, 
                            validation_data=(test_images, test_labels), verbose=1)
        end_time = time.time()
    
    return history, end_time - start_time

<p style='text-align: justify;'> The next step is to perform the model training. Note that in the step below, we will use the GPU to train the model and then the CPU to train and compare their execution times. (Depending on the GPU and CPU of your machine, this step may take some time). </p>

In [ ]:
gpu_history, gpu_time = train_model('/GPU:0', train_images, train_labels)
print(f"Training time on GPU: {gpu_time:.2f} seconds")

### ☆ Solution `CIFAR-100` using TensorFlow on CPU and GPU ☆

#### ⊗ Importing Packages

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import time

#### ⊗ Verify the devices

It is very important, before trying to execute anything on any device, to verify if it is available and if TensorFlow can use it.

#####  Checking the environmental availability

In [ ]:
# Checking if GPU is available
print(f"CPU device: ", tf.config.list_physical_devices('CPU'))
print(f"GPU device: ", tf.config.list_physical_devices('GPU'))

#### ⊗ Downloading the dataset

Now we need to download the CIFAR-100 dataset to be able to make predictions. This dataset is a labeled images, meaning that each image to be loaded already has a known label.

In [ ]:
# Loading the CIFAR-100 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar100.load_data()

#### ⊗ Normalizing the dataset

After downloading the entire set of images, we need to normalize them so that we can use them in our example.

In [ ]:
# Normalizing pixel values to the [0, 1] range
train_images, test_images = train_images / 255.0, test_images / 255.0

#### ⊗ Training the model

<p style='text-align: justify;'>
 Below we have the training function with model creation and model compilation. Notice that we need to get the creation and compilation together with the training because we need to set in which device everything will be done. We need to do this because if we don't set the device to create and compile the model, the TensorFlow will choose the faster device, in this case, the GPU, so if we try to use the CPU to train the model, it will fail because the model will be created on the GPU. 
</p>

In [ ]:
def train_model(device, train_images, train_labels):
    with tf.device(device):
        
        # Creating the CNN model
        model = models.Sequential([
            layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.Flatten(),
            layers.Dense(64, activation='relu'),
            layers.Dense(100)
        ])

        # Compiling the model
        model.compile(optimizer='adam',
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])

        start_time = time.time()
        history = model.fit(train_images, train_labels, epochs=10, 
                            validation_data=(test_images, test_labels), verbose=1)
        end_time = time.time()
    
    return history, end_time - start_time

<p style='text-align: justify;'> 
    The next step is to train the model. Note that in the step below we will use the CPU to train the model. 
</p>

In [ ]:
history, cpu_time = train_model('/CPU:0', train_images, train_labels)
print(f"\nCPU Training time: {cpu_time:.2f} seconds or ({cpu_time / 60:.2f} minutes)")

Now we will perform the same training only using the GPU.

In [ ]:
history, gpu_time = train_model('/GPU:0', train_images, train_labels)
print(f"\nGPU Training time: {gpu_time:.2f} seconds or ({gpu_time / 60:.2f} minutes)")

Now we will evaluate the speedup by comparing the GPU and CPU execution times.

In [ ]:
print(f"\nSpeedup:{cpu_time / gpu_time: .2f}X") 

### Comments about the results

<p style='text-align: justify;'>
We explored training neural networks with TensorFlow, comparing CPU and GPU performance on the CIFAR-10 and CIFAR-100 dataset using 10 epochs. When training with CIFAR-10 and utilizing the CPU, and GPU environments, the process can be executed in approximately:
</p>

|  TensorFlow |      CIFAR-10    |  CIFAR-100 |
|----------|:-------------:      |-----------:|
| CPU         |  811.17          |   0,000.00|
| GPU         |  37.23           |   00.00 |
| Speedup     |  21.78X          |   0.00 |

<p style='text-align: justify;'>
This outcome illustrates that the GPU has achieved nearly a <b>Speedup of 7X</b> compared to the CPU when running with 10 epochs in the algorithm with the highest computational cost (CIFAR-100). Thanks to its parallel computing capabilities, the GPU has substantially enhanced the training speed, which is particularly advantageous for handling extensive data and intricate models in deep learning.
</p>   

## Summary
In this notebook we have shown: 

- Install and use TensorFlow using GPU environments,
- Comparative performance tests between CPU and GPU on model training.

## Clear the memory
Before moving on, please execute the following cell to clear up the CPU memory. This is required to move on to the next notebook.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Next

In this section, you learned how to use TensorFlow in a simple example using a GPU environment. In the next section, you will learn about other applications in which those devices can be pretty useful, in the notebbok [_03-hpc-simulations-pytorch.ipynb_](03-hpc-simulations-pytorch.ipynb).